<a href="https://colab.research.google.com/github/iapetus509/M5-methods/blob/master/20241120_M5_top7P_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

m5_forecasting_accuracy_path = kagglehub.competition_download('m5-forecasting-accuracy')

print('Data source import complete.')

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 참고할만한 visual적인 대회설명 및 코드¶
# https://s-saci95.medium.com/machine-learning-for-retail-sales-forecasting-features-engineering-4edfee7c9cbc

# 해당코드. TOP 7% 코드임.
# Feature Engineering: drived parameter값인 lag값 생성
# 아래처럼 lag 7, 28을 쓰면,  TOP 7% (390등, 0.668) 결과값이 나옴


# 참고코드. Top 3% (170등, 0.61)
# https://www.kaggle.com/competitions/m5-forecasting-accuracy/discussion/163154
# https://github.com/btrotta/kaggle-m5
# TOP 3% code is quite short (<300 lines) and uses only fairly basic features in a LightGBM model.  I didn't use any "magic" adjustment factors.
# TOP 7% 처럼 3년데이터를 사용했고 (노이즈를 줄이고 계절적 추세를 포착하기 위해), 실제로 모델을 훈련시키는 데 1년의 데이터를 사용했습니다.
# 추가로 크리스마스의 영향 때문에 훈련 기간에서 12 월을 제외함.
# TOP 7%은 7, 28 Lag지만, TOP 3%는 7, 14, 28 lag 평균을 사용함.


# 참고코드. Top 1 (0.51844,  Privit 0.54284)
# https://www.kaggle.com/code/urkoallende/m5-lgbm-improving-top1-solution-p-s-0-51844
# I've simplified the approach, using only one dataset to train all models, as well as only using a dynamic step training approach.
# To get a private score below 0.520, you will need to change the step size before training
# STEPS = list(np.arange(2,30,2)) - That will not train in time unless you're using GPU support or doing it locally.


# 'tweedie'/'poisson' objectives are making better optimization.
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/144067#)
# Tweedie Gradient Boosting for Extremely Unbalanced Zero-inflated Data : https://arxiv.org/pdf/1811.10192.pdf .
# Bioprocess에서 Bioburden과 같이 Zero-Inflated data가 많아서.. 전엔 ZINB 기법을 사용했었는데, 예측모델링 기법에 "Tweedie" 기법을 활용할 수도 있겠음.


# 참고코드. LSTM모델

# 참고코드. Kalman filter

# M5 및 time-serise 동영상.
# https://youtu.be/pVLEo-IvlxY?si=ZbFPgVZipW7XdppD&t=5159


In [ ]:
calendar = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
sales = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")
# train_validation = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv") # Public 점수를 위한거니 필요없음

train_evaluation = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv")
#train_evaluation = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv", nrows=500) #순서대로 500개
# train_evaluation = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv").sample(500, random_state=42) #랜덤 500개

submission = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sample_submission.csv")

In [ ]:
calendar

In [ ]:
sales

In [ ]:
train_evaluation

In [ ]:
submission

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
# 달력 데이터 전처리 함수
def prep_calendar(df):
    # 사용하지 않는 열 제거
    df = df.drop(["date", "weekday", "event_type_1", "event_type_2"], axis=1)

    # d 열 값에서 'd_' 제거 후 정수형으로 변환
    df = df.assign(d=df.d.str[2:].astype(int))

    # 결측값이 많은 event_name 열들을 전처리
    to_ordinal = ["event_name_1", "event_name_2"]
    df[to_ordinal] = df[to_ordinal].fillna("1")
    df[to_ordinal] = OrdinalEncoder(dtype="int").fit_transform(df[to_ordinal]) + 1

    # 메모리 절약을 위해 int8로 변환
    to_int8 = ["wday", "month", "snap_CA", "snap_TX", "snap_WI"] + to_ordinal
    df[to_int8] = df[to_int8].astype("int8")

    return df
calendar = prep_calendar(calendar)
calendar

In [ ]:
# # 데이터 전처리 함수  (아래에 한줄 한줄 debugging 진행)
# def prep_data(df, drop_d):
#     # 초기 수요 기록을 제거해 분석 기간을 현재 시점과 가깝게 유지
#     df = df.drop(["d_" + str(i+1) for i in range(drop_d)], axis=1)
#     df = df.assign(id=df.id.str.replace("_evaluation", ""))

#     # 예측할 기간을 위한 열 추가
#     df = df.reindex(columns=df.columns.tolist() + ["d_" + str(FIRST + i) for i in range(LENGTH)])

#     # 데이터를 long 형식으로 변환 (일자별 수요로 변환)
#     df = df.melt(id_vars=["id", "item_id", "store_id", "state_id", "dept_id", "cat_id"], var_name='d', value_name='demand')

#     # 일자와 수요를 정수형으로 변환
#     df = df.assign(d=df.d.str[2:].astype("int16"),
#                    demand=df.demand.astype("float32"))

#     # 수요 예측 특성 생성
#     df = demand_features(df)

#     # 결측치가 있는 행 제거
#     df = df[df.d > (drop_d + max(LAGS) + max(WINDOWS))]

#     # 달력 정보 및 판매 가격을 일자별로 결합
#     df = df.merge(calendar, how="left", on="d")
#     df = df.merge(sales, how="left", on=["store_id", "item_id", "wm_yr_wk"])
#     df = df.drop(["wm_yr_wk"], axis=1)
#     # 범주형 변수를 정수형으로 인코딩
#     for v in ["item_id", "store_id", "state_id", "dept_id", "cat_id"]:
#         df[v] = OrdinalEncoder(dtype="int").fit_transform(df[[v]]).astype("int16") + 1

#     # 예측 대상인 demand와 고유값인 id, d 열은 입력 변수에서 제외
#     x = list(set(df.columns) - {'id', 'd', 'demand'})

#     # 테스트 데이터 분리
#     test = df[df.d >= FIRST - max(LAGS) - max(WINDOWS) - LENGTH]
#     df = df[df.d < FIRST]

#     # 학습 데이터를 훈련 및 검증 세트로 분할
#     xtrain, xvalid, ytrain, yvalid = train_test_split(df[x], df["demand"], test_size=0.1, shuffle=True, random_state=42)
#     train = lgb.Dataset(xtrain, label=ytrain)
#     valid = lgb.Dataset(xvalid, label=yvalid)
#     return train, valid, test, x
# train, valid, test, x = prep_data(train_evaluation, 1000 - LENGTH)
# test.head()

In [ ]:
LAGS = [7, 28]
WINDOWS = [7, 28]
FIRST = 1942  # 테스트 데이터의 첫 번째 날
LENGTH = 28  # 테스트 기간
# 시계열 특성(수요 예측을 위한 lag 및 rolling mean)을 생성하는 함수
def demand_features(df):
    for lag in LAGS:
        # 과거 수요를 lag만큼 지연하여 생성
        df[f'lag_t{lag}'] = df.groupby('id')['demand'].transform(lambda x: x.shift(lag)).astype("float32")
        for w in WINDOWS:
            # 과거 수요의 rolling mean 생성
            df[f'rolling_mean_lag{lag}_w{w}'] = df.groupby('id')[f'lag_t{lag}'].transform(lambda x: x.rolling(w).mean()).astype("float32")
    return df
from sklearn.model_selection import train_test_split


In [ ]:
df = train_evaluation

In [ ]:
# 약 5년의 데이터 중 최근 1000일 (3년) 정도만 모델에 사용. 오래된것은 안맞으니깐..

drop_d = 1000 - LENGTH
df = df.drop(["d_" + str(i+1) for i in range(drop_d)], axis=1)
df

In [ ]:
# id 에서 쓸모 없었던 _evaluation 삭제진행
df = df.assign(id=df.id.str.replace("_evaluation", ""))
df

In [ ]:
# 예측할 기간을 위한 열 추가  (d_1941 이후 28일인 d_1969 생성)
df = df.reindex(columns=df.columns.tolist() + ["d_" + str(FIRST + i) for i in range(LENGTH)])
df

In [ ]:
df.dtypes

In [ ]:
# 데이터를 long (stack) 형식으로 변환 (일자별 수요로 변환)
df = df.melt(id_vars=["id", "item_id", "store_id", "state_id", "dept_id", "cat_id"], var_name='d', value_name='demand')
df

In [ ]:
df.dtypes

In [ ]:
# 일자와 수요를 정수형으로 변환
df = df.assign(d=df.d.str[2:].astype("int16"), demand=df.demand.astype("float32"))
df

In [ ]:
df.dtypes

In [ ]:
 # 수요 예측 특성 생성  (맨위에 demand_features쪽에 lag7, lag28, lag7_mean, lag28_mean 만든것 넣기)
df = demand_features(df)
df

In [ ]:
# 결측치가 있는 행 제거
df = df[df.d > (drop_d + max(LAGS) + max(WINDOWS))]
df

In [ ]:
# 달력 정보 및 판매 가격을 일자별로 결합
df = df.merge(calendar, how="left", on="d")
df

In [ ]:
# on에 있는 값을 기준으로 Sales 에 있는 sell_price 추가
df = df.merge(sales, how="left", on=["store_id", "item_id", "wm_yr_wk"])
df

In [ ]:
# 중복 변수 삭제
df = df.drop(["wm_yr_wk"], axis=1)

In [ ]:
# 범주형 변수를 정수형으로 인코딩
for v in ["item_id", "store_id", "state_id", "dept_id", "cat_id"]:
    df[v] = OrdinalEncoder(dtype="int").fit_transform(df[[v]]).astype("int16") + 1

In [ ]:
# 예측 대상인 demand와 고유값인 id, d 열은 입력 변수에서 제외
x = list(set(df.columns) - {'id', 'd', 'demand'})

In [ ]:
# 테스트 데이터 분리
test = df[df.d >= FIRST - max(LAGS) - max(WINDOWS) - LENGTH]
test

In [ ]:
# 테스트 데이터 분리
df = df[df.d < FIRST]
df

In [ ]:
##### 학습
import lightgbm as lgb

In [ ]:
# 학습 데이터를 훈련 및 검증 세트로 분할
xtrain, xvalid, ytrain, yvalid = train_test_split(df[x], df["demand"], test_size=0.1, shuffle=True, random_state=2024)
train = lgb.Dataset(xtrain, label=ytrain)
valid = lgb.Dataset(xvalid, label=yvalid)

In [ ]:
# 학습진행

def fit_model(train, valid):
    params = {
        'metric': 'rmse',
        'objective': 'poisson',   # twiddle
        'seed': 2024,
        'force_row_wise': True,   # 순서대로
        'learning_rate': 0.08,
        'lambda': 0.1,
        'num_leaves': 63,
        'sub_row': 0.7,
        'bagging_freq': 1,
        'colsample_bytree': 0.7
    }
    fit = lgb.train(params, train, num_boost_round=2000, valid_sets=[valid])  #early stop (괜히 더 떨어질 수 있음. test set에 적합한 숫자를 넣은것일 수 있음)
    return fit
fit = fit_model(train, valid)

In [ ]:
# 제출

def demand_features_eval(df):
    return df.groupby('id', sort=False).last().reset_index()

def pred_all(fit, test, x):
    for day in range(FIRST, FIRST + LENGTH):
        test_day = demand_features_eval(test[(test.d <= day) & (test.d >= day - max(LAGS) - max(WINDOWS))])
        test.loc[test.d == day, "demand"] = fit.predict(test_day[x])
    return test

def pred_to_csv(test, cols, file):
    test = test.assign(id=test.id + "_" + np.where(test.d < FIRST, "validation", "evaluation"),
                       F="F" + (test.d - FIRST + LENGTH + 1 - LENGTH * (test.d >= FIRST)).astype("str"))
    submission = test.pivot(index="id", columns="F", values="demand").reset_index()[cols].fillna(1)
    submission.to_csv(file, index=False)

pred = pred_all(fit, test, x)
pred_to_csv(pred, cols=submission.columns, file="submission.csv")

In [ ]:
# 시각화 1

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
def analyze_validation_predictions(pred_df, item_id_to_analyze=None):
    """
    validation 데이터에 대한 예측 성능을 분석하고 시각화하는 함수

    Parameters:
    pred_df: 예측이 포함된 데이터프레임
    item_id_to_analyze: 분석할 특정 제품 ID (None인 경우 전체 데이터에 대한 성능 계산)
    """
    # validation 데이터만 필터링 (FIRST 이전 데이터)
    val_data = pred_df[pred_df.d < FIRST].copy()

    # 전체 데이터에 대한 성능 지표 계산
    y_true = val_data['demand']
    y_pred = val_data['demand_pred']  # 예측값 열 이름이 다를 수 있음에 주의

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print(f"전체 Validation 데이터 성능:")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2 Score: {r2:.4f}")
    if item_id_to_analyze is not None:
    # 특정 제품에 대한 데이터 필터링
        item_data = val_data[val_data.item_id == item_id_to_analyze]

        # 특정 제품에 대한 성능 지표 계산
        item_rmse = np.sqrt(mean_squared_error(item_data['demand'],
                                             item_data['demand_pred']))
        item_r2 = r2_score(item_data['demand'], item_data['demand_pred'])

        print(f"\n제품 ID {item_id_to_analyze}에 대한 성능:")
        print(f"RMSE: {item_rmse:.4f}")
        print(f"R2 Score: {item_r2:.4f}")

        # 시각화
        plt.figure(figsize=(12, 6))
        plt.plot(item_data['d'], item_data['demand'],
                label='Actual', color='blue', alpha=0.7)
        plt.plot(item_data['d'], item_data['demand_pred'],
                label='Predict', color='red', alpha=0.7)

        plt.title(f'Item ID {item_id_to_analyze} - Actual and Predicted demand')
        plt.xlabel('Date (d)')
        plt.ylabel('Demand')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

# 산점도 그래프로 실제값과 예측값의 관계 시각화
        plt.figure(figsize=(8, 8))
        plt.scatter(item_data['demand'], item_data['demand_pred'],
                   alpha=0.5, color='blue')
        plt.plot([0, item_data['demand'].max()],
                [0, item_data['demand'].max()],
                'r--', alpha=0.8)
        plt.title(f'Item ID {item_id_to_analyze} - Actual and Predicted demand')
        plt.xlabel('Actual')
        plt.ylabel('Predict')
        plt.grid(True, alpha=0.3)
        plt.show()

# 사용 예시
# 먼저 예측값 열을 추가합니다
pred['demand_pred'] = pred['demand']  # 실제 예측값으로 수정 필요
# 전체 데이터에 대한 성능 분석
analyze_validation_predictions(pred)
# 특정 제품(예: item_id=1)에 대한 분석과 시각화
analyze_validation_predictions(pred, item_id_to_analyze=1)